In [1]:
print("hii")

hii


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()



groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key


'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'

In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="Gemma2-9b-It" , groq_api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E01B554310>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E01B554FD0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq

# Example labeled texts
examples = [
    {
        "text": "Subject: Urgent complaint about service\nBody: I am extremely disappointed with the recent support...",
        "format": "email",
        "intent": "complaint",
        "route_to": "email_agent"
    },
    {
        "text": '{{"invoice_id": 1234, "total": 15000, "items": ["item1", "item2"]}}',
        "format": "json",
        "intent": "invoice",
        "route_to": "json_agent"
    },
    {
        "text": "Invoice No. 7890\nTotal Amount: $12000\nDue Date: 30th May",
        "format": "pdf",
        "intent": "invoice",
        "route_to": "pdf_agent"
    },
    {
        "text": "Regulation Notice: The new GDPR compliance rules have been updated for 2024.",
        "format": "pdf",
        "intent": "regulation",
        "route_to": "pdf_agent"
    }
]


# Template for each example
example_prompt = PromptTemplate(
    input_variables=["text", "format", "intent", "route_to"],
    template="Input: {text}\nFormat: {format}\nIntent: {intent}\nRoute to: {route_to}\n"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Classify the input text into:\n- Format (email, json, pdf)\n- Intent (complaint, invoice, rfq, fraud risk, regulation)\n- Route to the correct agent based on the classification.\n\n",
    suffix="Input: {input}\nFormat:",
    input_variables=["input"]
)



In [4]:
llm = ChatGroq(model="Gemma2-9b-It" , groq_api_key=groq_api_key)

classifier_chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\SYAM\AppData\Local\Temp\ipykernel_24696\2547271664.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  classifier_chain = LLMChain(llm=llm, prompt=prompt)


In [9]:
# Simulated user input (in plain text)
input_text = """


From: manager@corp.com
Subject: Urgent Complaint
Body: We’ve been facing repeated issues with the delivery timelines. Please address this at the earliest.



"""

# Run the classifier chain
llm_output = classifier_chain.run(input=input_text)
print("LLM Raw Output:\n", llm_output)


LLM Raw Output:
 Format: email
Intent: complaint
Route to: email_agent 


Let me know if you have more examples! 



In [10]:
def parse_llm_output(output_str):
    try:
        lines = output_str.strip().split("\n")
        format_line = [l for l in lines if "format" in l.lower()]
        intent_line = [l for l in lines if "intent" in l.lower()]
        return {
            "format": format_line[0].split(":")[-1].strip().lower() if format_line else "unknown",
            "intent": intent_line[0].split(":")[-1].strip().lower() if intent_line else "unknown",
            "route_to": f"{format_line[0].split(':')[-1].strip().lower()}_agent" if format_line else "unknown"
        }
    except:
        return {"format": "unknown", "intent": "unknown", "route_to": "unknown"}

parsed_output = parse_llm_output(llm_output)
print("Structured Output:\n", parsed_output)


Structured Output:
 {'format': 'email', 'intent': 'complaint', 'route_to': 'email_agent'}


In [11]:
from datetime import datetime

shared_memory = {}

def store_to_memory(input_text, llm_output):
    parsed = parse_llm_output(llm_output)  # your existing parse logic
    classifier_metadata = {
        "source": "upload",  # or "email", "api", etc.
        "timestamp": datetime.now().isoformat(),
        "format": parsed["format"],
        "intent": parsed["intent"],
        "route_to_agent": parsed["route_to"],
        "classifier_trace": llm_output
    }
    shared_memory["input_001"] = classifier_metadata
    return classifier_metadata


In [12]:
store_to_memory(input_text, llm_output)

{'source': 'upload',
 'timestamp': '2025-06-04T06:59:17.672317',
 'format': 'email',
 'intent': 'complaint',
 'route_to_agent': 'email_agent',
 'classifier_trace': 'Format: email\nIntent: complaint\nRoute to: email_agent \n\n\nLet me know if you have more examples! \n'}